На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue. Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [31]:
from math import asin
from typing import Union
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [32]:
df = pd.read_excel('gb_sem_8_hm.xlsx')
df.head(10)



,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.00
1,2423,control,0.00
2,9411,control,0.00
3,7311,control,0.00
4,6174,variant,0.00
5,2380,variant,0.00
6,2849,control,0.00
7,9168,control,0.00
8,6205,variant,0.00
9,7548,control,0.00


In [33]:
df

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.00
1,2423,control,0.00
2,9411,control,0.00
3,7311,control,0.00
4,6174,variant,0.00
...,...,...,...
9995,1981,control,0.00
9996,502,variant,0.00
9997,9214,variant,0.00
9998,7741,control,0.00


In [34]:
df.shape

(10000, 3)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [36]:
df.describe()

,USER_ID,REVENUE
count,"10,000.00","10,000.00"
mean,"4,981.08",0.10
std,"2,890.59",2.32
min,2.00,0.00
25%,"2,468.75",0.00
50%,"4,962.00",0.00
75%,"7,511.50",0.00
max,"10,000.00",196.01


In [37]:
df.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [38]:
df.USER_ID.nunique()

6324

In [39]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [40]:
df.shape

(7865, 3)

In [41]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [42]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [43]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)
     

In [44]:
df_new.describe()

,USER_ID,REVENUE
count,"4,783.00","4,783.00"
mean,"4,994.40",0.14
std,"2,898.62",3.01
min,2.00,0.00
25%,"2,476.00",0.00
50%,"4,975.00",0.00
75%,"7,515.00",0.00
max,"9,998.00",196.01


In [45]:
df.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [46]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [47]:
control = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [48]:
continious_result(control, treatment, column='REVENUE')

100%|██████████| 10000/10000 [00:08<00:00, 1207.49it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.04,0.16,0.71,"[-0.003, 0.317]",0.12


In [49]:
control.describe()

,USER_ID,REVENUE
count,"2,390.00","2,390.00"
mean,"5,020.88",0.20
std,"2,904.85",4.17
min,2.00,0.00
25%,"2,517.25",0.00
50%,"5,012.50",0.00
75%,"7,616.00",0.00
max,"9,998.00",196.01


In [50]:
treatment.describe()

,USER_ID,REVENUE
count,"2,393.00","2,393.00"
mean,"4,967.94",0.07
std,"2,892.75",0.86
min,4.00,0.00
25%,"2,435.00",0.00
50%,"4,955.00",0.00
75%,"7,379.00",0.00
max,"9,995.00",23.04


вывод:

доверительный интервал [-0.005, 0.323 ] содержит ноль, alpha и beta большие 5%, отклонения в группах незначительны.

## Рекомендация
Статистические значемые отклонения не значительны. Нововведения не принесут желаемого результата. 